In [1]:
from header import *
import psutil
from mne.stats import spatio_temporal_cluster_1samp_test,spatio_temporal_cluster_test
from scipy.stats.distributions import f,t
from tqdm import tqdm
import visbrain
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
t0 = time.perf_counter()
task = 'SMEG' #'MIMOSA'
states = ['RS','FA','OM']
subjects = get_subjlist(task)
subjects = subjects[:subjects.index('109')]

reject = ['072']
for sub in reject:
    if sub in subjects:
        subjects.remove(sub)

subjects.sort()
#subjects = ['002', '007', '012', '014', '018', '032', '037', '042', '083', '087', '089', '091', '050', '055', '064', '067', '068', '070', '073']
experts = []
novices = []
experts_i = []
novices_i = []
for s,sub in enumerate(subjects):
    if expertise(sub) == 'N':
        novices.append(sub)
        novices_i.append(s)
    if expertise(sub) == 'E':
        experts.append(sub)
        experts_i.append(s)

In [3]:
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[0]
noise_cov = 'baseline_cov'
stc_ext = '-lh.stc'
sfreq = 200
start = None
stop = None
stc_path = op.join(Analysis_path, task, 'meg', 'SourceEstimate')

In [4]:
#for n,name in enumerate(names):
for st,state in enumerate(states):
    print(state)
    for su,sub in enumerate(tqdm(subjects)):
        stc_file = op.join(stc_path, 'fsaverage', sub, state+'-'+name+'-'+noise_cov+'*'+stc_ext)
        file = glob.glob(stc_file)[0]
        data = mne.read_source_estimate(file[:file.index(stc_ext)])
        if sfreq:
            data.resample(sfreq)
        if not st and not su:#and not n 
            times = data.times
            if not start:
                start = times[0]
            if not stop:
                stop = times[-1]
            vertices = np.concatenate([['lh_' + str(x) for x in data.lh_vertno],['rh_' + str(x) for x in data.rh_vertno]])
            try:
                del stc
            except:
                pass
            stc = np.zeros((len(states), len(subjects), data.data.shape[1], data.data.shape[0]))
            stc = xr.DataArray(stc, dims=['state', 'subject', 'time', 'src'], coords={'state':states, 'subject':subjects, 'time':times, 'src':vertices})
        stc[st,su] = data.data.T
        del data

  0%|          | 0/35 [00:00<?, ?it/s]

RS


  0%|          | 0/35 [00:00<?, ?it/s]

FA


  0%|          | 0/35 [00:00<?, ?it/s]

OM


100%|██████████| 35/35 [05:03<00:00,  8.67s/it]


In [5]:
def cluster_perm_test(X1, X2, stat_file, test_key, times, vertices, mode='a', p_threshold=0.01, connectivity='ico5', paired=False):
    """
    If paired, test X2-X1.
    Input: arrays of shape (subjects, time, space)
    """
    if type(connectivity) is str:
        connectivity = mne.spatial_tris_connectivity(mne.grade_to_tris(int(connectivity[-1])))
    
    if paired:
        t_threshold = -t.ppf(p_threshold / 2, (X2-X1).shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = spatio_temporal_cluster_1samp_test((X2-X1), connectivity=connectivity, threshold=t_threshold, n_jobs=4)
    else:
        f_threshold = f.ppf(1 - p_threshold / 2, X1.shape[0] - 1, X2.shape[0] - 1)
        T_obs, clusters, cluster_pv, H0 = spatio_temporal_cluster_test([X1,X2], connectivity=connectivity, threshold=f_threshold, n_jobs=4)
    
    p_val = np.ones(T_obs.shape)
    clu_inds = np.zeros(T_obs.shape)
    
    for c,clu in enumerate(clusters):
        p_val[clu] = cluster_pv[c]
        clu_inds[clu] = c+1
    
    stats = np.zeros((3, 2, len(times), len(vertices)))
    stats = xr.DataArray(stats, dims=['data', 'hemisphere', 'time', 'src'], coords={'data':['T_stat', 'p_val', 'clu_inds'], 'hemisphere':['left', 'right'], 'time':times, 'src':vertices})
    for h,hemi in enumerate(['left', 'right']):
        i = h + 1
        h *= len(vertices)
        i *= len(vertices)
        stats.loc['T_stat', hemi] = T_obs[:, h:i]
        stats.loc['p_val', hemi] = p_val[:, h:i]
        stats.loc['clu_inds', hemi] = clu_inds[:, h:i]
    
    if not op.isdir(op.split(stat_file)[0]):
        os.makedirs(op.split(stat_file)[0])
    
    if not op.isfile(stat_file):
        mode = 'w'
    
    stats.to_netcdf(path=stat_file, group=test_key, mode=mode)
    #for h,hemi in enumerate(['lh', 'rh']):
    #    i = h + 1
    #    h *= T_obs.shape[1]/2
    #    i *= T_obs.shape[1]/2
    #    
    #    np.savetxt(stat_file+'-'+hemi+'-T_stat.tsv', T_obs[:,int(h):int(i)], delimiter='\t')
    #    np.savetxt(stat_file+'-'+hemi+'-p_val.tsv', mask[:,int(h):int(i)], delimiter='\t')
    #
    return T_obs, clusters, cluster_pv, H0, mask

In [6]:
spacing='ico5'
connectivity = mne.spatial_tris_connectivity(mne.grade_to_tris(int(spacing[-1])))

-- number of connected vertices : 20484


In [7]:
T_obs=dict(); clusters=dict(); cluster_pv=dict(); H0=dict(); mask=dict()

In [8]:
t0 = time.perf_counter()
ti = t0
start = .35#float(stc.time[0])
stop = .5#float(stc.time[-1])
vertno = np.arange(stc.shape[-1]/2, dtype=int)

stat_path = op.join(Analysis_path, task, 'meg', 'Stats', '{}_{}'.format(round(start,4),round(stop,4)))
if not op.isdir(stat_path):
    os.makedirs(stat_path)
stat_file = op.join(stat_path, name+'-'+noise_cov+'.nc')

test_key = 'RS_vs_FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',:,start:stop].values, stc.loc['FA',:,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',:,start:stop].values, stc.loc['OM',:,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'OM_vs_FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['OM',:,start:stop].values, stc.loc['FA',:,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_FA+E'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',experts,start:stop].values, stc.loc['FA',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_OM+E'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',experts,start:stop].values, stc.loc['OM',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'OM_vs_FA+E'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['OM',experts,start:stop].values, stc.loc['FA',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_FA+N'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',novices,start:stop].values, stc.loc['FA',novices,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'RS_vs_OM+N'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',novices,start:stop].values, stc.loc['OM',novices,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'OM_vs_FA+N'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['OM',novices,start:stop].values, stc.loc['FA',novices,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity, paired=True)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()


test_key = 'N_vs_E+RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['RS',novices,start:stop].values, stc.loc['RS',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+FA'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['FA',novices,start:stop].values, stc.loc['FA',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['OM',novices,start:stop].values, stc.loc['OM',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+FA-RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['FA',novices,start:stop].values-stc.loc['RS',novices,start:stop].values, stc.loc['FA',experts,start:stop].values-stc.loc['RS',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+OM-RS'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['OM',novices,start:stop].values-stc.loc['RS',novices,start:stop].values, stc.loc['OM',experts,start:stop].values-stc.loc['RS',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()

test_key = 'N_vs_E+FA-OM'
T_obs[test_key], clusters[test_key], cluster_pv[test_key], H0[test_key], mask[test_key] = cluster_perm_test(stc.loc['FA',novices,start:stop].values-stc.loc['OM',novices,start:stop].values, stc.loc['FA',experts,start:stop].values-stc.loc['OM',experts,start:stop].values, stat_file=stat_file, test_key=test_key, times=stc.time.loc[start:stop], vertices=vertno, connectivity=connectivity)
print('\t',test_key, time.perf_counter()-ti,'\n\t*****\n')
ti = time.perf_counter()


T = ti - t0
print('\n*****\nTotal running time:', T)

stat_fun(H1): min=-5.431127 max=5.500841
Running initial clustering
Found 1536 clusters
Permuting 1023 times...
[...................................     ] 87.84314 \    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  2.9min remaining:  2.9min


[....................................... ] 99.22481 |    

[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  3.3min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  3.3min finished


Computing cluster p-values
Done.
	 RS_vs_FA 202.19359459599946 
	*****

stat_fun(H1): min=-6.827243 max=5.303220
Running initial clustering
Found 1641 clusters
Permuting 1023 times...
[..................................      ] 86.82171 \    

...

Computing cluster p-values
Done.
	 N_vs_E+OM-RS 659.5115441179951 
	*****

stat_fun(H1): min=0.000000 max=32.486025
Running initial clustering
Found 4161 clusters
Permuting 1023 times...
[...................................     ] 87.84314 \    

[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed: 10.4min remaining: 10.4min


[....................................... ] 99.22481 |    

[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 10.9min remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed: 10.9min finished


Computing cluster p-values
Done.
	 N_vs_E+FA-OM 658.3602882909472 
	*****


*****
Total running time: 5531.146985872008
